In [167]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import time
import psycopg2
import os
import json
import re

In [168]:
def create_connection():
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    cursor = conn.cursor()

    return conn, cursor

In [169]:
def create_fb_data_table():
    #create connection to db
    conn, cursor = create_connection()

    table_check_sql = """
    SELECT EXISTS (
        SELECT FROM pg_tables 
        WHERE schemaname = 'public' 
        AND tablename = 'fb_data'
    );
    """

    create_table_sql = """
            CREATE TABLE fb_data (
                id SERIAL PRIMARY KEY,
                text TEXT NOT NULL,
                source TEXT NOT NULL,
                UNIQUE (text)
            );
            """
    try:
        cursor.execute(table_check_sql)
        table_exists = cursor.fetchone()[0]
        if not table_exists:
            cursor.execute(create_table_sql)
            conn.commit()
            print("Table created successfully")
        else:
            print("Table exists")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    cursor.close()
    conn.close()

In [170]:
def get_facebook_posts(username, post_count_break):
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    driver.get(f"https://www.facebook.com/{username}/posts")

    keywords = {
        "election", "vote", "voting", "candidate", "campaign",
        "politician", "politics", "government", "congress", "senate", 
        "house of representatives", "white house", "president", 
        "vice president", "democrat", "republican", "liberal", "conservative",
        "policy", "legislation", "bill", "law", "executive order", "constitution",
        "supreme court", "judiciary", "governor", "mayor", "local government",
        "senator", "representative", "parliament", "prime minister", "ambassador",
        "diplomacy", "foreign policy", "domestic policy", "budget", "taxes", "taxation",
        "healthcare", "immigration", "border", "national security", "military",
        "war", "conflict", "peace", "international relations", "trade", "sanctions",
        "lobbying", "political party", "activism", "protest", "movement", "rights",
        "freedom", "civil rights", "human rights", "reform", 
        "corruption", "scandal", "ethics", "voter suppression", "gerrymandering",
        "climate change", "environmental policy", "energy policy", "education policy",
        "economic policy", "labor", "wages", "social security", "medicare", "medicaid",
        "welfare", "public assistance", "subsidy", "entitlement", "budget deficit",
        "debt ceiling", "public debt", "federal reserve", "central bank", "inflation",
        "unemployment", "job creation", "economic growth", "GDP", "trade deficit",
        "export", "import", "tariff", "free trade", "fair trade", "monetary policy",
        "fiscal policy", "regulation", "deregulation", "privatization", "nationalization",
        "kamala", "trump", "vance", "walz", "democrats", "republicans"
    }

    insert_sql = """
    INSERT INTO fb_data (
        text, source
    ) VALUES (
        %s, %s
    )
    """

    conn,cursor = create_connection()
    conn.autocommit = True
    post_count = 0

    while post_count < post_count_break:
        elements = driver.find_elements(By.XPATH, '//div[@data-ad-preview="message"]')
        for element in elements:
            content = element.text

            found_keywords = []
            for keyword in keywords:
                pattern = r'\b' + re.escape(keyword) + r'\b'
                if re.search(pattern, content, re.IGNORECASE):
                    found_keywords.append(keyword)


            if found_keywords:
                try:
                    cursor.execute(insert_sql,(content, username))
                    post_count+=1
                except(Exception, psycopg2.DatabaseError) as error:
                    print(error)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    driver.quit()
    conn.close()
    cursor.close()    

In [171]:
username = "nytimes"  
post_count_break = 10

create_fb_data_table()
get_facebook_posts(username, post_count_break)


Table exists
duplicate key value violates unique constraint "fb_data_text_key"
DETAIL:  Key (text)=(Democrats sued the Georgia state election board on Monday, arguing that measures approved by the board this month seeking to alter the election certification process in the state were illegal and could create chaos on Election Day.) already exists.

duplicate key value violates unique constraint "fb_data_text_key"
DETAIL:  Key (text)=(Colorado Republicans voted to remove Dave Williams, a hard-line state representative who was elected to helm the Colorado Republican Party, amid mounting criticism of his leadership, including over recent posts that attacked the LGBTQ community.) already exists.

duplicate key value violates unique constraint "fb_data_text_key"
DETAIL:  Key (text)=(Democrats sued the Georgia state election board on Monday, arguing that measures approved by the board this month seeking to alter the election certification process in the state were illegal and could create cha